# 自然言語処理を用いた質問応答

## 1.1 自然言語処理における深層学習
自然言語処理において深層学習は下記のような様々なタスクで使われる
- 文章を異なる言語に翻訳する機械翻訳
- 重要な情報だけを抽出する自動要約
- 文書と質問を元に回答する機械読解
- 画像に関する質問に応答するシステム
- etc

これらはいずれも世界中の研究者・有名IT企業が精力的に取り組んでいる分野であり、</br>
これらに用いられる最新の手法にはほぼ必ずと言っていいほど深層学習が用いられている</br>
よって、自然言語処理関連の研究・ビジネスをしようとしている人にとって、深層学習・ニューラルネットワークを学ぶことはほぼ必須

なぜ自然言語処理で深層学習がこれほど使われているかというと、</br>
単語をコンピュータで扱うためにはどうしても数値に変換する必要ある</br>
その古典的な方法として、
- One hot vector
- TFIDF

などを「自然言語処理」講座で学んだが、</br>
実際これらは手軽に行えるので手始めに自然言語処理で何かしたい時には最適な手法である</br>
<b>しかし、これらのベクトルには、以下の問題がある</b>

そこでニューラルネットワークのモデルを使うと、誤差逆伝播法によって単語のベクトルを学習できるため、</br>
各単語にわずか数百次元のベクトルを割り当てる（Embeddingと言う）だけでよく、</br>
さらに文脈を考慮しながら単語のベクトルを学習できるので、単語の意味が似ているものは似たようなベクトルが得られるなど、</br>
TFIDFなどに比べて単語の意味を扱えると言った利点がある


## 1.2 Embedding
Embeddingとは日本語で「埋め込み」という意味</br>
深層学習による自然言語処理では、単語という記号をdd次元ベクトル（ddは100〜300程度）に埋め込む、Embeddingという処理を行う</br>
単語を扱うニューラルネットワークのモデルを構築する際、一番始めに行うのがこのEmbedding</br>
tensorflow.kerasではEmbedding Layerというものが用意されており、以下のように使える

ここで必ず指定しなければならない値が、
- input_dim: 語彙数、単語の種類の数
- output_dim: 単語ベクトルの次元
- input_length: 各文の長さ

全単語の単語ベクトルを結合したEmbedding Matrixと呼ばれるものの例が以下</br>
前提として各単語に特有のIDを割り振り、そのIDがEmbedding Matrixの何行目になるのかを指す （ID=0が0行目、ID=iがi行目）</br>
そして各単語のベクトルがEmbedding Matrixの行に相当</br>
Embedding Matrixの横の長さdは単語ベクトルの次元に相当</br>

![](images/Embedding_Matrix.jpeg)

各セルの値はtensorflow.kerasが自動でランダムな値を格納</br>
図のように、多くの場合0行目にはunk、すなわちUnknown（未知語）を割り当てる</br>
unkを使う理由は、扱う語彙を限定してその他のマイナーな単語は全てUnknownとすることでメモリを節約するため</br>
語彙の制限の仕方は、扱うコーパス（文書）に出現する頻度が一定以上のものだけを扱うなどが一般的

- Caution
    - Embeddingへの入力は単語に割り当てたIDからなる行列で、 サイズは（バッチサイズ、文の長さ）である必要がある
      バッチサイズとは、一度に並列して計算を行うデータ（文の）数を表す
    - 文の長さは全てのデータで統一する必要がある
      ここで問題になるのが、文の長さは一定ではないということ
        - そこで恣意的に文の長さをDとし、
            - 長さD以下の文は長さがDになるよう0を追加する
            - 長さD以上の文は長さがDになるよう末尾から単語を削る


In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ

# 本来は単語をIDに変換する必要がありますが、今回は簡単に入力データを準備しました。
input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

# modelにEmbeddingを追加
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))

# input_dataのshapeがどのように変わるのか確認してください。
output = model.predict(input_data)
print(output.shape)

(32, 20, 100)


2021-12-29 22:22:18.153636: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


## 1.3 RNN
RNNとはRecurrent Neural Networkの略称で、日本語で「再帰ニューラルネット」</br>
可変長系列、すなわち任意の長さの入力列を扱うことに優れており、自然言語処理において頻繁に使われる重要な機構</br>
言語以外でも、時系列データであれば使えるので、音声認識など活用の幅は広い

Recurrentは「繰り返しの」という意味</br>
つまりRNNとは「繰り返しの」ニューラルネットワークという意味

![](images/rnn_math.png)

![](images/rnn_image.png)

tensorflow.kerasを使うときにこれらの厳密な定義を覚えておく必要はない</br>
入力列を順番に入力していき、各時刻で隠れ状態ベクトルと出力が得られるということを覚えておくこと</br>
RNNにも他のニューラルネットと同様に複数の層を重ねることができる


## 1.4 LSTM

### 1.4.1 LSTMとは
LSTMとはRNNの一種で、 LSTMはRNNに特有な欠点を補う機構を持っている

- RNN特有の欠点
  RNNは時間方向に深いニューラルネットなので、初期の方に入力した値を考慮してパラメータを学習させるのが難しい
  つまり、長期記憶(long-term memory)が苦手
  感覚的に言うと、RNNは初めの方に入力された要素を「忘れて」しまう

上記の欠点を補うための機構として有名なのがLSTM</br>
LSTMとはLong Short-Term Memoryの略称で、その名の通り長期記憶と短期記憶を可能にするもの</br>
LSTMは1.3章のRNNに「ゲート」という概念を導入したもので、ゲート付きRNNの一種</br>
このゲートによって長期記憶と短期記憶が可能

![](images/lstm_image.png)


### 1.4.2 LSTMの実装
早速tensorflow.kerasでLSTMを実装していく</br>
tensorflow.kerasにはLSTMを簡単に使うことができるモジュールが用意されているため、</br>
数式を意識することなくLSTMを使うことができる</br>

```python
from tensorflow.keras.layers import LSTM
units = 200
lstm = LSTM(units, return_sequences=True)
```

units: LSTMの隠れ状態ベクトルの次元数であり、大抵100から300程度の値</br>
一般に学習すべきパラメータの数は多いほど複雑な現象をモデル化できるが、その分学習させるのが大変（消費メモリが増える、学習時間が長い）

return_sequences: LSTMの出力の形式をどのようにするかを決めるための引数</br>
return_sequencesがTrueなら、LSTMは全ての入力系列に対応する出力系列（隠れ状態ベクトルh_1〜h_T）を出力</br>
return_sequencesがFalseなら、LSTMは最後の時刻TTにおける隠れ状態ベクトルのみを出力</br>
後の章で全ての出力系列を使うことになるので、ここではreturn_sequencesをTrueにしておく

モデルの構築方法は1.2章で学んだEmbeddingと繋げると以下のようになる
```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(lstm_units, return_sequences=True))
```

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

# 今回も簡単に入力データを準備しました。
input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

# modelにLSTMを追加してください。
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))
model.add(LSTM(lstm_units, return_sequences=True))


# input_dataのshapeがどのように変わるのか確認してください。
output = model.predict(input_data)
print(output.shape)

(32, 20, 200)


### 1.4.3 BiLSTM
LSTM含めRNNに入力系列x={x_1, ..., x_T}をx_1からx_Tにかけて先頭から順に入力</br>
x_Tからx_1にかけて後ろから順に入力して行くこともできる</br>
さらに2つの入力させる向きを組み合わせた双方向再帰ニューラルネット（bi-directional recurrent neural network）がよく用いられる</br>
利点は、各時刻において先頭から伝播してきた情報と後ろから伝播してきた情報、すなわち入力系列全体の情報を考慮できること</br>
2方向のLSTMを繋げたものをBidirectional LSTM、通称BiLSTMと言う

![](images/BiLSTM_image.png)
2つの向きのRNNを繋げる方法はいくつかあるが</br>
LSTMを引数にすることで簡単に実装できる

```
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
bilstm = Bidirectional(LSTM(units, return_sequences=True), merge_mode='sum')
```

- merge_mode
  2方向のLSTMをどう繋げるかを指定するためのもので、 基本的に{'sum', 'mul', 'concat', 'ave'}の中から選ぶ
    - sum: 要素和
    - mul: 要素積
    - concat: 結合
    - ave: 平均
    - None: 結合せずにlistを返す

![](images/image_merge_mode.png)


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

# 今回も簡単に入力データを準備しました。
input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)
print(input_data.shape)
# modelにBiLSTMを追加してください。
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))
model.add(Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='sum'))


# input_dataのshapeがどのように変わるのか確認してください。
output = model.predict(input_data)
print(output.shape)

(32, 20)
(32, 20, 200)


## 1.5 Softmax関数
Softmax関数は活性化関数の一種で、クラス分類を行う際にニューラルネットの一番出力に近い層で使われる

![](images/Softmax.png)

実際にtensorflow.kerasで実装するときは、以下のように、バッチごとにsoftmax関数を適用して使う
```python
from tensorflow.keras.layers import Activation

# xのサイズ: [バッチサイズ、クラス数]
y = Activation('softmax')(x)
# sum(y[0]) = 1, sum(y[1]) = 1, ...
```

これはActivation('softmax')のデフォルトの設定が</br>
入力xxのサイズの最後の要素、クラス数の軸方向にsoftmaxを作用させるというものだから</br>
つまり、xxのサイズが[バッチサイズ、d、クラス数]のように3次元であってもActivation('softmax')を適用可能</br>

note: tensorflow.keras.models.Sequentialを使わずにこのようにモデルを記述する方法をFunctional APIと言う


In [5]:
import numpy as np
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model


x = Input(shape=(20, 5))
# xにsoftmaxを作用させてください
y = Activation('softmax')(x)

model = Model(inputs=x, outputs=y)

sample_input = np.ones((12, 20, 5))
sample_output = model.predict(sample_input)

print(np.sum(sample_output, axis=2))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


### 1.6 Attention
### 1.6.1 Attentionとは
今2つの文章s={s_1, ... , s_N}, t={t_1, ... , t_L}があるとする</br>
ここでは仮にssを質問文とし、ttをそれに対する回答文の候補だとする</br>
この時、機械に自動でtがsに対する回答文として妥当かどうか判断させるにはどのようにしたら良いか？</br>
tだけをいくら眺めても、tが妥当かどうかは分からない</br>
sの情報を参照しつつ、tが妥当かどうか判断する必要がある</br>

<b>そこでAttention Mechanism（注意機構）という機構が役に立つ</b>

これまでの章で文をRNNによって隠れ状態ベクトルに変換できることを学んできた</br>
具体的には2つの別々のRNNを用意し、一方のRNNによってsを隠れ状態ベクトルh^{(s)}={h_1^{(s)},...,h_N^{(s)}}に変換し、</br>
もう一方のRNNによってtを隠れ状態ベクトルh^{(t)}={h_1^{(t)},...,h_L^{(t)}}に変換できる</br>

そこでsの情報を考慮してtの情報を使うために、以下のようにtの各時刻においてsの各時刻の隠れ状態ベクトルを考慮した特徴を計算する</br>
![](images/Attention.png)

図には単方向のRNNの場合を示しましたが、双方向のRNNであってもAttentionは適用可能

![](images/Attention_image.png)

このAttentionという機構は深層学習による自然言語処理では当たり前のように使われる重要な技術で、</br>
機械翻訳や、自動要約、対話の論文で頻繁に登場する</br>
歴史的には機械翻訳に初めて使われて以来その有用性が広く認知されるようになった</br>

また、今回はsの隠れ状態ベクトルの重み付き平均を用いる、Soft Attentionを紹介したが、</br>
ランダムに1つの隠れ状態ベクトルを選択する、Hard Attentionも存在する</br>
さらにそこから派生して、画像認識の分野でも使われることがある</br>
中でもGoogleが発表したAttention is all you needという論文はとても有名


### 1.6.2 Attentionの実装
tensorflow.kerasでAttentionを実装するためには、Mergeレイヤーを使う必要がある
tensorflow.kerasのバージョン2.0以降では前の章まで使っていたSequential ModelをMergeすることができないため、
ここではtensorflow.kerasのFunctional APIを使う。Functional APIの簡単な使い方は以下

Sequential Modelではただ新しいLayerをaddしていくだけでしたが、
Functional APIを使うことでもっと自在に複雑なモデルを組むことができる

```
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 文1の長さ
seq_length2 = 30 # 文2の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

input1 = Input(shape=(seq_length1,))
embed1 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length1)(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)

input2 = Input(shape=(seq_length2,))
embed2 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length2)(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)

# 要素ごとの積を計算する
product = dot([bilstm2, bilstm1], axes=2) # productのサイズ：[バッチサイズ、文2の長さ、文1の長さ]

a = Activation('softmax')(product)
c = dot([a, bilstm1], axes=[2, 1])
c_bilstm2 = concatenate([c, bilstm2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_bilstm2)

model = Model(inputs=[input1, input2], outputs=h)
```

このように各Layerを関数のように使うのでFunctional APIと呼ばれる</br>
また新しく登場したInputレイヤーで指定するshapeにはbatchサイズを入れないよう注意</br>
そしてModelを定義するときは引数にinputsとoutputsを指定する必要があるが</br>
入力や出力が複数ある場合はリストに入れて渡せば大丈夫</br>
そして新しく登場したdot([u, v], axes=2)は、uとvのバッチごとの行列積を計算</br>
指定したaxesにおける次元数はuとvで等しくなければならない</br>
また、dot([u, v], axes=[1,2])とするとuの1次元とvの2次元、のように別々の次元の指定も可能

![](images/Attention_impl.png)




In [7]:
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 文1の長さ
seq_length2 = 30 # 文2の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

# 2つのLSTMに共通のEmbeddingLayerを使うため、はじめにEmbeddingLayerを定義します。
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)

# 要素ごとの積を計算する
product = dot([bilstm2, bilstm1], axes=2) # サイズ：[バッチサイズ、文2の長さ、文1の長さ]

# ここにAttention mechanismを実装してください
a = Activation('softmax')(product)
c = dot([a, bilstm1], axes=[2, 1])
c_bilstm2 = concatenate([c, bilstm2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_bilstm2)

model = Model(inputs=[input1, input2], outputs=h)

sample_input1 = np.arange(batch_size * seq_length1).reshape(batch_size, seq_length1)
sample_input2 = np.arange(batch_size * seq_length2).reshape(batch_size, seq_length2)

sample_output = model.predict([sample_input1, sample_input2])
print(sample_output.shape)

(32, 30, 200)


2021-12-30 22:27:50.739291: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


## 1.7 Dropout
Dropoutとは訓練時に変数の一部をランダムに0に設定することによって、汎化性能を上げ、過学習を防ぐための手法

- 過学習とは
    - ニューラルネットなどのモデルで教師あり学習を行う場合、しばしば訓練データに適合しすぎて、
      検証データでのパフォーマンスが訓練データに比べて著しく下がる「過学習」を起こしてしまう
- 汎化性能とは
    - 訓練データで過学習することなく、訓練データと検証データに関わらず一般的に高いパフォーマンスができることを「汎化性能」が高いと言う
      実際に使うときは、変数のうち0に設定する割合を0から1の間の値で設定して、Dropoutレイヤーを追加する

```python
# Sequentialモデルを使う場合
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

model = Sequential()
...
model.add(Dropout(0.3))

# Functional APIを使う場合
from tensorflow.keras.layers import Dropout
y = Dropout(0.3)(x)
```

In [8]:
import numpy as np
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model

batch_size = 32  # バッチサイズ
vocab_size = 1000  # 扱う語彙の数
embedding_dim = 100  # 単語ベクトルの次元
seq_length = 20  # 文1の長さ
lstm_units = 200  # LSTMの隠れ状態ベクトルの次元数

input = Input(shape=(seq_length,))

embed = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=seq_length)(input)

bilstm = Bidirectional(
    LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed)

output = Dropout(0.3)(bilstm)

model = Model(inputs=input, outputs=output)

sample_input = np.arange(
    batch_size * seq_length).reshape(batch_size, seq_length)

sample_output = model.predict(sample_input)

print(sample_output.shape)

(32, 20, 400)


# 回答文選択システムの実装

## 2.1 回答文選択システム
実践編では、基礎編で学んだことを活かして回答文選択システムを実装</br>
質問文に対して、回答文の候補がいくつか与えられて、その中から正しい回答文を自動で選択するシステム</br>
用いるデータセットはAllen AIのTextbook Question Answeringというもの

1. 分かち書き
2. 文字の正規化
3. 単語のID化
4. (自然言語処理で深層学習を使う場合) Padding(= 入力の全ての文の長さを統一)


## 2.2 データの前処理
### 2.2.1 正規化・分かち書き
英語の正規化については、今回は最も基本的な大文字または小文字に統一という処理のみ扱う
```python
s = "I am Darwin."
s = s.lower()
print(s)
# => "i am darwin."
```

次は分かち書き。英語の分かち書きに用いられるツールの一つにnltkというものがある</br>
nltkでは分かち書きの他にも見出し語化、語幹化なども使えますが、今回は簡単のため分かち書きのみを使う</br>
```python
from nltk.tokenize import word_tokenize
t = "he isn't darwin."
t = word_tokenize(t)
print(t)
# => ['he', 'is', "n't", 'darwin', '.']
```

In [5]:
import json
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

with open("./tqa_train_val_test/train.json") as f:
    train = json.load(f)

# trainはリストで、各要素に質問と回答の候補、答えが辞書型のデータとして格納されています。
# train[0] = {'answerChoices': {'a': 'solid Earth.',
#  'b': 'Earths oceans.',
#  'c': 'Earths atmosphere.',
#  'd': 'all of the above'},
# 'correctAnswer': 'd',
# 'question': 'Earth science is the study of'}

target = train[0]["question"]

# 小文字に統一してください
target = target.lower()

# 分かち書きをしてください
target = word_tokenize(target)

print(target)

['earth', 'science', 'is', 'the', 'study', 'of']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2.2.2 単語のID化
単語のままではニューラルネットに入力として与えられないので、IDに変換する必要がある</br>
ここでIDとはEmbedding Matrixの行に相当</br>
また、データに登場する単語全てにIDを付与すると、全体の語彙数が膨大になってしまう場合が多くある</br>

そこで、頻度が一定以上の単語のみにIDを与え、データをIDの列に変換


In [6]:
import json
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

with open("./tqa_train_val_test/train.json", "r") as f:
    train = json.load(f)

def preprocess(s):
    s = s.lower()
    s = word_tokenize(s)
    return s

sentences = []
for t in train:
    q = t['question']
    q = preprocess(q)
    sentences.append(q)
    for i, a in t['answerChoices'].items():
        a = preprocess(a)
        sentences.append(a)

vocab = {}
for s in sentences:
    for w in s:
        # 必要な処理を行ってください
        # print(w)
        vocab[w] = vocab.get(w, 0) + 1

word2id = {}
word2id['<unk>'] = 0
for w, v in vocab.items():
    if not w in word2id and v >= 2:
        # 必要な処理を行ってください
        word2id[w] = len(word2id)

target = preprocess(train[0]["question"])
target = [word2id.get(w, 0) for w in target]
print(target)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[1, 2, 3, 4, 5, 6]


### 2.2.3 Padding
深層学習をする際、文章など長さがバラバラなデータはそのままでは行列演算ができないため、
強制的に末尾にダミーIDの0を追加したり、文末から必要なだけ単語を削除したりする
padding（とtruncating）を入力データに対して行う必要がある

tensorflow.kerasにはそのための便利な関数が用意されているので今回はそれを使う
```python
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
s = [[1,2], [3,4,5], [6,7,8], [9,10,11,12,13,14]]
s = pad_sequences(s, maxlen=5, dtype=np.int32, padding='post', truncating='post', value=0)
print(s)
# => array([[ 1,  2,  0,  0,  0],
#       [ 3,  4,  5,  0,  0],
#       [ 6,  7,  8,  0,  0],
#       [ 9, 10, 11, 12, 13]], dtype=int32)
```

このようにpaddingとtruncatingを行った上で、numpy配列にして返してくれる
- maxlen: 統一する長さ
- dtype: データの型
- padding: 'pre'か'post'を指定し、前と後ろのどちらにpaddingするかを決める
- truncating: 'pre'か'post'を指定し、前と後ろのどちらをtruncatingするか決める
- value: paddingするときに用いる値


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 引数にはこれを使ってください。
maxlen = 10
dtype = np.int32
padding = 'post'
truncating = 'post'
# データ
s = [[1,2,3,4,5,6], [7,8,9,10,11,12,13,14,15,16,17,18], [19,20,21,22,23]]
# padding, truncatingをしてください。
s = pad_sequences(s, maxlen=maxlen, dtype=dtype, padding=padding, truncating=truncating, value=value)
print(s)

## 2.3 Attention-based QA-LSTM
### 2.3.1 全体像
回答文選択システムを実装していく</br>
学習モデルには Attention-based QA-LSTMというものを分かりやすく改良したものを使う</br>

1. QuestionとAnswerを別々にBiLSTMに入力
2. QuestionからAnswerに対してAttentionをし、Questionを考慮したAnswerの情報を得る
3. Questionの各時刻の隠れ状態ベクトルの平均をとって(mean pooling)ベクトルqを得る
   一方でQuestionからAttentionを施した後、Answerの各時刻の隠れ状態ベクトルの平均をとってベクトルaを得る
4. この2つのベクトルを[q;a;|q-a|;q*a][q;a;∣q−a∣;q∗a]のようにq, a, |q-a|∣q−a∣, q*aベクトルを結合して、
   順伝播ニューラルネット、Softmax関数を経て2つのユニットからなる出力

この結合の仕方はFacebook researchが発表したInferSentという有名な手法を参考にしている</br>
このモデルの出力層はユニットが2つありますが、正解の回答文については[1,0]を、不正解の回答文については[0,1]を予測するように学習していく

![](images/Attention-based_QA-LSTM.png)

### 2.3.2 質問と回答のBiLSTM
Bidirectional LSTM(BiLSTM)とは、固有表現を認識する際に、後ろから読むことで左右両方向の文脈情報を捉えることが可能</br>
QuestionとAnswerのBiLSTMを実装

![](images/Bidirectional_LSTM.png)


In [1]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model


vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 質問の長さ
seq_length2 = 10 # 回答の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)
h1 = Dropout(0.2)(bilstm1)
model1 = Model(inputs=input1, outputs=h1)


input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)
h2 = Dropout(0.2)(bilstm2)
model2 = Model(inputs=input2, outputs=h2)

model1.summary()
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding (Embedding)       multiple                  100000    
                                                                 
 bidirectional (Bidirectiona  (None, 20, 400)          481600    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 20, 400)           0         
                                                                 
Total params: 581,600
Trainable params: 581,600
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Sh

### 2.3.3 質問から回答へのAttention
Attention modelの実装</br>
QuestionからAnswerへのAttentionであることに注意して作成する</br>

![](images/Attention_model.png)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 質問の長さ
seq_length2 = 10 # 回答の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)
h1 = Dropout(0.2)(bilstm1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)
h2 = Dropout(0.2)(bilstm2)

# 要素ごとの積を計算する
product = dot([h2, h1], axes=2) # サイズ：[バッチサイズ、回答の長さ、質問の長さ]
a = Activation('softmax')(product)
c = dot([a, h1], axes=[2, 1])
c_h2 = concatenate([c, h2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_h2)
"""
sample code
product = dot([bilstm2, bilstm1], axes=2) # サイズ：[バッチサイズ、文2の長さ、文1の長さ]
a = Activation('softmax')(product)
c = dot([a, bilstm1], axes=[2, 1])
c_bilstm2 = concatenate([c, bilstm2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_bilstm2)
"""

model = Model(inputs=[input1, input2], outputs=h)
model.summary()

### 2.3.4 出力層、コンパイル
mean poolingから出力層までを実装</br>
最後にsoftmax関数を使うことに注意すること

xのサイズ: [batch_size, steps, features]</br>
yのサイズ: [batch_size, downsampled_steps, features]


In [7]:
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate, subtract, multiply
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 質問の長さ
seq_length2 = 10 # 回答の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = lstm_units * 2 # 最終出力のベクトルの次元数

def abs_sub(x):
    return K.abs(x[0] - x[1])

embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)
h1 = Dropout(0.2)(bilstm1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)
h2 = Dropout(0.2)(bilstm2)

# 要素ごとの積を計算する
product = dot([h2, h1], axes=2) # サイズ：[バッチサイズ、回答の長さ、質問の長さ]
a = Activation('softmax')(product)
c = dot([a, h1], axes=[2, 1])
c_h2 = concatenate([c, h2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_h2)

mean_pooled_1 = AveragePooling1D(pool_size=seq_length1, strides=1, padding='valid')(h1)
mean_pooled_2 = AveragePooling1D(pool_size=seq_length2, strides=1, padding='valid')(h)

mean_pooled_1 = Reshape((lstm_units * 2,))(mean_pooled_1)
mean_pooled_2 = Reshape((lstm_units * 2,))(mean_pooled_2)

sub = Lambda(abs_sub)([mean_pooled_1, mean_pooled_2])
mult = multiply([mean_pooled_1, mean_pooled_2])
con = concatenate([mean_pooled_1, mean_pooled_2, sub, mult], axis=-1)
con = Reshape((lstm_units * 2 * 4,))(con)
output = Dense(2, activation='softmax')(con)

model = Model(inputs=[input1, input2], outputs=output)
model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        multiple             100000      ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 20, 400)     481600      ['embedding_1[0][0]']      

## 2.4 訓練
modelの構築が終わったらmodelの学習をする</br>
padding以外の前処理を全て終えてIDに変換したものを./5050_nlp_data/に置いてあるため、それを利用する</br>
単語をIDに変換するための辞書は./5050_nlp_data/word2id.jsonに保存してある</br>
ファイル名は訓練データが./5050_nlp_data/preprocessed_train.json, 検証データが./5050_nlp_data/preprocessed_val.json

例:
```json
{'answerChoices': {'a': [1082, 1181, 586, 2952, 0],
  'b': [1471, 2492, 773, 0, 1297],
  'c': [811, 2575, 0, 1181, 2841, 0],
  'd': [2031, 1984, 1099, 0, 3345, 975, 87, 697, 1366]},
 'correctAnswer': 'a',
 'question': [544, 0]}
```

In [8]:
import json
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open("./tqa_train_val_test/word2id.json", "r") as f:
    word2id = json.load(f)

batch_size = 500 # バッチサイズ
vocab_size = len(word2id) # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 質問の長さ
seq_length2 = 10 # 回答の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)
h1 = Dropout(0.2)(bilstm1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)
h2 = Dropout(0.2)(bilstm2)

# 要素ごとの積を計算する
product = dot([h2, h1], axes=2) # サイズ：[バッチサイズ、回答の長さ、質問の長さ]
a = Activation('softmax')(product)
c = dot([a, h1], axes=[2, 1])
c_h2 = concatenate([c, h2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_h2)

mean_pooled_1 = AveragePooling1D(pool_size=seq_length1, strides=1, padding='valid')(h1)
mean_pooled_2 = AveragePooling1D(pool_size=seq_length2, strides=1, padding='valid')(h)
con = concatenate([mean_pooled_1, mean_pooled_2], axis=-1)
con = Reshape((lstm_units * 2 + hidden_dim,))(con)
output = Dense(2, activation='softmax')(con)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(optimizer="adam", loss="categorical_crossentropy")

with open("./5050_nlp_data/preprocessed_train.json", "r") as f:
    train = json.load(f)

questions = []
answers = []
outputs = []
for t in train:

    for i, ans in t["answerChoices"].items():
        if i == t["correctAnswer"]:
            outputs.append([1, 0])
        else:
            outputs.append([0, 1])
        # 以下のコードを埋めてください
        questions.append(t["question"])
        answers.append(ans)

questions = pad_sequences(questions, maxlen=seq_length1, dtype=np.int32, padding='post', truncating='post', value=0)
answers = pad_sequences(answers, maxlen=seq_length2, dtype=np.int32, padding='post', truncating='post', value=0)
outputs = np.array(outputs)

# 学習させています
model.fit([questions[:10*100], answers[:10*100]], outputs[:10*100], batch_size=batch_size)
# ローカルで作業する場合は以下のコードを実行してください。

#　model.save_weights("./5050_nlp_data/model.hdf5")
#　model_json = model.to_json()

#　with open("./5050_nlp_data/model.json", "w") as f:
    #　json.dump(model_json, f)

FileNotFoundError: [Errno 2] No such file or directory: './5050_nlp_data/word2id.json'

## 2.5 テスト
最後に検証データを使ってテスト</br>
2値分類なので、精度は正解率(Accuracy)、適合率(Precision)、再現率(Recall)を計算すること
また、こちらで5epoch学習させた学習済みモデル("./5050_nlp_data/trained_model.hdf5")を用意したので、それを利用する

In [ ]:
import json
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences


with open("./5050_nlp_data/preprocessed_val.json", "r") as f:
    val = json.load(f)
seq_length1 = 20 # 質問の長さ
seq_length2 = 10 # 回答の長さ

questions = []
answers = []
outputs = []
for t in val:
    for i, ans in t["answerChoices"].items():
        if i == t["correctAnswer"]:
            outputs.append([1, 0])
        else:
            outputs.append([0, 1])
        questions.append(t["question"])
        answers.append(ans)

questions = pad_sequences(questions, maxlen=seq_length1, dtype=np.int32, padding='post', truncating='post', value=0)
answers = pad_sequences(answers, maxlen=seq_length2, dtype=np.int32, padding='post', truncating='post', value=0)

with open("./5050_nlp_data/model.json", "r") as f:
    model_json = json.load(f)
model = model_from_json(model_json)
model.load_weights("./5050_nlp_data/trained_model.hdf5")

pred = model.predict([questions, answers])

pred_idx = np.argmax(pred, axis=-1)
true_idx = np.argmax(outputs, axis=-1)

pred_idx = pred_idx[:1000] # 検証データが大きいため、最初の１０００個のデータのみを使用
ture_idx = true_idx[:1000] # 検証データが大きいため、最初の１０００個のデータのみを使用

TP = 0
FP = 0
FN = 0
TN = 0

# 以下にコードを入力してください。
for p, t in zip(pred_idx, true_idx):
    if p == 0 and t == 0:
        TP += 1

    elif p == 0 and t == 1:
        FP += 1

    elif p == 1 and t == 0:
        FN += 1

    else:
        TN += 1


print("正解率:", (TP+TN)/(TP+FP+FN+TN))
print("適合率:", TP/(TP+FP))
print("再現率:", TP/(TP+FN))

## 2.6 Attentionの可視化
Attentionでは、文章sから文章tへのAttentionを施すにあたり、</br>
以下のようにsのj番目の単語がtのi番目の単語にどれくらい注目しているかをa_{ij} が表していると言える

![](images/visualize_attention.png)

このa_{ij}を(i,j)(i,j)成分に持つような行列AをAttention Matrixと呼ぶ</br>
Attention Matrixを見ればssとttの単語間にどのような関係があるのかを可視化できる</br>
質問単語(横軸)と回答単語(縦軸)で関係が深いものは白く表示される

In [9]:
import matplotlib.pyplot as plt
import json
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import model_from_json
import mpl_toolkits.axes_grid1

batch_size = 32  # バッチサイズ
embedding_dim = 100  # 単語ベクトルの次元
seq_length1 = 20  # 質問の長さ
seq_length2 = 10  # 回答の長さ
lstm_units = 200  # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200  # 最終出力のベクトルの次元数

with open("./5050_nlp_data/preprocessed_val.json", "r") as f:
    val = json.load(f)

questions = []
answers = []
outputs = []
for t in val:
    for i, ans in t["answerChoices"].items():
        if i == t["correctAnswer"]:
            outputs.append([1, 0])
        else:
            outputs.append([0, 1])
        questions.append(t["question"])
        answers.append(ans)

questions = pad_sequences(questions, maxlen=seq_length1,
                          dtype=np.int32, padding='post', truncating='post', value=0)
answers = pad_sequences(answers, maxlen=seq_length2,
                        dtype=np.int32, padding='post', truncating='post', value=0)

with open("./5050_nlp_data/word2id.json", "r") as f:
    word2id = json.load(f)

vocab_size = len(word2id)  # 扱う語彙の数
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(
    LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)
h1 = Dropout(0.2)(bilstm1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(
    LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)
h2 = Dropout(0.2)(bilstm2)


# 要素ごとの積を計算する
product = dot([h2, h1], axes=2)  # サイズ：[バッチサイズ、回答の長さ、質問の長さ]
a = Activation('softmax')(product)

c = dot([a, h1], axes=[2, 1])
c_h2 = concatenate([c, h2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_h2)

mean_pooled_1 = AveragePooling1D(
    pool_size=seq_length1, strides=1, padding='valid')(h1)
mean_pooled_2 = AveragePooling1D(
    pool_size=seq_length2, strides=1, padding='valid')(h)
con = concatenate([mean_pooled_1, mean_pooled_2], axis=-1)
con = Reshape((lstm_units * 2 + hidden_dim,))(con)
output = Dense(2, activation='softmax')(con)

# ここを解答してください
prob_model = Model(inputs=[input1, input2], outputs=[a, output])

prob_model.load_weights("./5050_nlp_data/trained_model.hdf5")

question = np.array([[2945, 1752, 2993, 1099, 122, 2717, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
answer = np.array([[2841, 830, 2433, 0, 0, 0, 0, 0, 0, 0]])

att, pred = prob_model.predict([question, answer])

id2word = {v: k for k, v in word2id.items()}

q_words = [id2word[w] for w in question[0]]
a_words = [id2word[w] for w in answer[0]]

f = plt.figure(figsize=(8, 8.5))
ax = f.add_subplot(1, 1, 1)

# add image
i = ax.imshow(att[0], interpolation='nearest', cmap='gray')

# add labels
ax.set_yticks(range(att.shape[1]))
ax.set_yticklabels(a_words)

ax.set_xticks(range(att.shape[2]))
ax.set_xticklabels(q_words, rotation=45)

ax.set_xlabel('Question')
ax.set_ylabel('Answer')

# add colorbar
divider = mpl_toolkits.axes_grid1.make_axes_locatable(ax)
cax = divider.append_axes('right', '5%', pad='3%')
plt.colorbar(i, cax=cax)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './5050_nlp_data/preprocessed_val.json'